In [2]:
import os
print(os.environ.get('CUDA_PATH'))

None


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import torch
import nncf
from nncf import NNCFConfig
from nncf.torch import create_compressed_model
from nncf.torch import register_default_init_args


/home/nguyen.thanh.huyenb/anaconda3/envs/yolo/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nguyen.thanh.huyenb/anaconda3/envs/yolo/lib/python3.10/site-packages/nncf/torch/__init__.py:23: UserWarning: NNCF provides best results with torch==1.9.1, while current torch version is 1.13.0 - consider switching to torch==1.9.1
  warnings.warn("NNCF provides best results with torch=={bkc}, "
/home/nguyen.thanh.huyenb/anaconda3/envs/yolo/lib/python3.10/site-packages/nncf/torch/dynamic_graph/patch_pytorch.py:163: UserWarning: Not patching unique_dim since it is missing in this version of PyTorch
  warnings.warn("Not patching {} since it is missing in this version of PyTorch".format(op_name))


In [3]:
from models.experimental import attempt_load
weight_path = "/home/nguyen.thanh.huyenb/yolov7/yolov7/runs/train/yolov79/weights/best.pt"
model = attempt_load(weight_path, map_location="cpu")
gs = max(int(model.stride.max()), 32) 
gs

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
IDetect.fuse


/home/nguyen.thanh.huyenb/anaconda3/envs/yolo/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642922335/work/aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


32

In [22]:
def collate_fn(batch):
    img, label, path, shapes = zip(*batch)  # transposed
    for i, l in enumerate(label):
        l[:, 0] = i  # add target image index for build_targets()
    return (torch.stack(img, 0)) #, torch.cat(label, 0), # path, shapes

In [23]:
class OPT:
    single_cls = False

opt = OPT()

In [24]:
from utils.datasets import create_dataloader
data_path = "/home/nguyen.thanh.huyenb/yolov7/training_minicoco_dataset/test"
imgsz = 320
batch_size = 2

dataloader = create_dataloader(
            data_path,
            imgsz,
            batch_size,
            gs,
            opt,
            pad=0.5,
            rect=True,
            collate_fn=collate_fn
        )[0]


Scanning '/home/nguyen.thanh.huyenb/yolov7/training_minicoco_dataset/test/labels.cache' images and labels... 10 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 10/10 [00:00<?, ?it/s]


In [25]:
batch = next(iter(dataloader))
len(batch)
batch[0].shape
batch[1].shape

2

torch.Size([3, 256, 352])

torch.Size([3, 256, 352])

In [26]:
nncf_config_dict = {
    "input_info": [
      {
        "sample_size": [1, 3, 320, 320]
        },
      # {
      #   "sample_size": [1, 6]
      # }
    ],
    "compression": [
    {
      "algorithm": "quantization",
      
    }
    ]
}
# Load a configuration file to specify compression
nncf_config = NNCFConfig.from_dict(nncf_config_dict)

In [27]:
nncf_config = register_default_init_args(nncf_config, dataloader)
nncf_config


NotImplementedError: By default it is assumed that the data loader used for initialize produces a tuple/list of (*model_input*, *ground_truth*) and that no special forward arguments have to be set during init. If this is not the case, then instead of your regular data loader you need to pass a specialized version of PTInitializingDataLoader that returns a general (args, kwargs) tuple for your model to be called with at each __next__ call.

In [28]:
compression_ctrl, model = create_compressed_model(model, nncf_config)

INFO:nncf:Wrapping module Model/Sequential[model]/Conv[0]/Conv2d[conv] by Model/Sequential[model]/Conv[0]/NNCFConv2d[conv]
INFO:nncf:Wrapping module Model/Sequential[model]/Conv[1]/Conv2d[conv] by Model/Sequential[model]/Conv[1]/NNCFConv2d[conv]
INFO:nncf:Wrapping module Model/Sequential[model]/Conv[2]/Conv2d[conv] by Model/Sequential[model]/Conv[2]/NNCFConv2d[conv]
INFO:nncf:Wrapping module Model/Sequential[model]/Conv[3]/Conv2d[conv] by Model/Sequential[model]/Conv[3]/NNCFConv2d[conv]
INFO:nncf:Wrapping module Model/Sequential[model]/Conv[4]/Conv2d[conv] by Model/Sequential[model]/Conv[4]/NNCFConv2d[conv]
INFO:nncf:Wrapping module Model/Sequential[model]/Conv[5]/Conv2d[conv] by Model/Sequential[model]/Conv[5]/NNCFConv2d[conv]
INFO:nncf:Wrapping module Model/Sequential[model]/Conv[6]/Conv2d[conv] by Model/Sequential[model]/Conv[6]/NNCFConv2d[conv]
INFO:nncf:Wrapping module Model/Sequential[model]/Conv[7]/Conv2d[conv] by Model/Sequential[model]/Conv[7]/NNCFConv2d[conv]
INFO:nncf:Wrappi

KeyError: 232

In [2]:
import torch

/home/tran.duc.trungb/miniconda3/envs/nncf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
torch.zeros(5, 1)

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])